In [51]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3
import pandas as pd
import requests
from datetime import datetime
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL
import awswrangler as wr
import cryptocompare

In [52]:
crypto_compare_key = "6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4"
cryptocompare.cryptocompare._set_api_key_parameter(crypto_compare_key)

'&api_key=6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4'

In [53]:
session = boto3.Session(profile_name='master')
dynamo = session.resource('dynamodb')
s3_client = session.client('s3')

In [54]:
list_of_coins = [
    "BTC",
    "ETH",
    "BUSD",
    "USDT",
    "XRP",
    "SOL",
    "BNB",
    "DOT",
    "SHIB",
    "LTC"
  ]
list_of_currencies = ['USD']

In [55]:
df = wr.s3.read_csv('s3://kosmobiker-masterproject/data/coin_list/table/', boto3_session=session)


In [58]:
tmp_df = df[df['Name'].isin(list_of_coins)][['Name', 'ContentCreatedOn']]
created_on = dict(zip(tmp_df.Name,tmp_df.ContentCreatedOn))
created_on

{'BTC': 1417635237,
 'DOT': 1593791455,
 'USDT': 1499420073,
 'BUSD': 1569227610,
 'BNB': 1501700700,
 'SHIB': 1618843684,
 'SOL': 1586353077,
 'XRP': 1428671962,
 'LTC': 1426004323,
 'ETH': 1438259812}

In [65]:
def get_historical_data(coin:str,
                        cur:str,
                        created_on:dict,
                        ts=int(datetime.now().timestamp()),
                        limit=2000):
    data = []
    done = False
    while not done:
        try:
            tmp_json = cryptocompare.get_historical_price_hour(coin, cur, limit=limit, toTs=ts)
            if ts < created_on[coin]:
                done = True
            else:
                data.extend(tmp_json)
                ts -= 3600000
                done = False
        except Exception as err:
            print(err)
            done = True
    return pd.DataFrame(data)

In [69]:
df = get_historical_data('ETH', "USD", created_on)